## Datafame in Pyspark

DataFrame is collection of named columns.

Very similar to pandas!

In [1]:
from pyspark import SparkContext
sc = SparkContext()


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
regressionDataFrame = spark.read.csv('Advertising.csv',header=True, inferSchema = True)

In [4]:
regressionDataFrame.show(5)  # similar to df.head()

+---+-----+-----+---------+-----+
|_c0|   TV|radio|newspaper|sales|
+---+-----+-----+---------+-----+
|  1|230.1| 37.8|     69.2| 22.1|
|  2| 44.5| 39.3|     45.1| 10.4|
|  3| 17.2| 45.9|     69.3|  9.3|
|  4|151.5| 41.3|     58.5| 18.5|
|  5|180.8| 10.8|     58.4| 12.9|
+---+-----+-----+---------+-----+
only showing top 5 rows



In [5]:
type(regressionDataFrame)

pyspark.sql.dataframe.DataFrame

In [6]:
regressionDataFrame = regressionDataFrame.drop('_c0')

In [7]:
regressionDataFrame.show(5)

+-----+-----+---------+-----+
|   TV|radio|newspaper|sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows



In [8]:
regressionDataFrame.columns

['TV', 'radio', 'newspaper', 'sales']

In [9]:
# show how many rows have a columns with above 100 TV, and how many don't
regressionDataFrame.groupBy(regressionDataFrame.TV > 100).count().show(5)

+----------+-----+
|(TV > 100)|count|
+----------+-----+
|      true|  130|
|     false|   70|
+----------+-----+



In [10]:
regressionDataFrame.count()

200

In [11]:
# data slicing, getting only the rows with >100 in the TV column
regressionDataFrame.filter(regressionDataFrame.TV > 100).show(5)

+-----+-----+---------+-----+
|   TV|radio|newspaper|sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|120.2| 19.6|     11.6| 13.2|
|199.8|  2.6|     21.2| 10.6|
+-----+-----+---------+-----+
only showing top 5 rows



In [12]:
# use the dot operator to grab single columns
regressionDataFrame.select(regressionDataFrame.TV > 100).show(5)

+----------+
|(TV > 100)|
+----------+
|      true|
|     false|
|     false|
|      true|
|      true|
+----------+
only showing top 5 rows



In [13]:
regressionDataFrame.describe()

DataFrame[summary: string, TV: string, radio: string, newspaper: string, sales: string]

In [14]:
from pyspark.sql.functions import mean, min, max

# showing the mean, min, and max of different columns
regressionDataFrame.select([mean('TV'), min('TV'), max('TV')]).show()

+--------+-------+-------+
| avg(TV)|min(TV)|max(TV)|
+--------+-------+-------+
|147.0425|    0.7|  296.4|
+--------+-------+-------+



In [15]:
# crosstab is here too, not so useful for solely numerical data
regressionDataFrame.crosstab('TV', 'radio').show()

+--------+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|TV_radio|0.0|0.3|0.4|0.8|1.3|1.4|1.5|1.6|1.9|10.0|10.1|10.6|10.8|11.0|11.6|11.7|11.8|12.0|12.1|12.6|13.9|14.0|14.3|14.5|14.7|14.8|15.4|15.5|15.8|15.9|16.0|16.7|16.9|17.0|17.2|17.4|18.1|18.4|1

In [16]:
regressionDataFrameRDD = sc.parallelize(regressionDataFrame, 4)

TypeError: cannot pickle '_thread.RLock' object

Using value counts: https://napsterinblue.github.io/notes/spark/sparksql/value_counts/

## Other Notes

- Spark dataFrames can be made by Pandas DataFrames
- not as straightforward though (you can do it all in one call)
- PySpark is not as good at visualization though (it's not the intention)

## How to Create UDF in PySpark

- Sometimes you have to make your own function

- You need to tell SQL that the Python function can be used to manipulate the dataframe

- that's what UDF does it registers your Python function

- the `udf()` comes from `pyspark.sql.functions`

- need to specify the py fn, and the return type using one of those used in SQL, defined by Spark

- then you can use the function like normal

- remember that `df.select` can also be used to update rows in the DF

## DataBricks

Supports SQL, Python, R, and Scala - all in the same notebook!